In [1]:
import sys
import nivapy
import qgrid
import pandas as pd

# Quick hack to import from relative folder
sys.path.insert(0, '../ndbview')
from ndbview import ndb_queries

# Testing queries for NDBView

`ndb_queries.py` contains functions used by the Flask end points in `ndbview.py` to query the NIVADATABASE. The code is structured so that, as far as possible, the application-specific code is all in `ndbview.py`, and `ndb_queries.py` provides a basic database API that can also be used from Jupyter. This is useful for testing, and will eventually be useful for [NivaPy](https://github.com/NIVANorge/nivapy) too.

## 1. Connect to NIVADATABASE

In [2]:
# Connect to db
ora_eng = nivapy.da.connect()

Connection successful.


## 2. User defined parameters

In [3]:
# Period of interest
st_dt = '1900-01-01'
end_dt = '2018-12-31'

# Include LOD flags ('<' or '>') in output?
lod_flags = False

Now complete **either** step 3.1 to query by projects, or skip ahead to step 3.2 to query the stations directly. **Choose either step 3.1 *or* step 3.2, not both**.

## 3.1. Query by project

### 3.1.1. Select projects of interest

In [4]:
# List projects
prj_df = ndb_queries.get_all_projects(ora_eng)

# Convert to qgrid
prj_grid = qgrid.QgridWidget(df=prj_df, show_toolbar=False)
prj_grid

QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

In [5]:
# Show selected projects
sel_prj = prj_grid.get_selected_df()
sel_prj

,project_id,o_number,project_name,project_description
118,3320,11488-03,RID- Hovedelver,None


### 3.1.2. Get stations for selected project(s)

In [6]:
# Get stations for selected project
stn_df = ndb_queries.get_project_stations(sel_prj, ora_eng)

# Convert to qgrid
stn_grid = qgrid.QgridWidget(df=stn_df, show_toolbar=False)

print 'Number of unique station IDs:', len(stn_df)
stn_grid

Number of unique station IDs: 15


QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

### 3.1.3. Refine station selection

Make a final selection of the porjects of interest (or select none to include all stations for these projects).

In [7]:
# Get selected stations
if len(stn_grid.get_selected_df()) == 0:
    sel_stn = stn_df
else:
    sel_stn = stn_grid.get_selected_df()

print 'Number of stations selected:', len(sel_stn)
sel_stn

Number of stations selected: 11


,station_id,station_code,station_name,station_type,longitude,latitude
1,40353,TELESKI,Skienselva,Elv,9.611000,59.199000
2,40354,VAGEOTR,Otra,Elv,7.953984,58.186992
3,40355,VESENUM,Numedalslågen,Elv,10.070000,59.086000
4,40356,Glomma5,Sarpefossen,Elv,11.134000,59.278000
6,40357,STREORK,Orkla,Elv,9.772994,63.200996
7,40358,FINEALT,Altaelva,Elv,23.286977,69.900992
8,40360,NOREVEF,Vefsna,Elv,13.238980,65.748998
9,40361,ROGEORR,Orreelva,Elv,5.528989,58.730993
10,40374,HOREVOS,Vosso(Bolstadelvi),Elv,6.000000,60.647992
11,62167,OSLEALN,Alna,Elv,10.791359,59.905091


## 3.2. Select stations directly

In [ ]:
# List stations
stn_df = ndb_queries.get_all_stations(ora_eng)

# Convert to qgrid
stn_grid = qgrid.QgridWidget(df=stn_df, show_toolbar=False)
stn_grid

In [ ]:
# Show selected projects
sel_stn = stn_grid.get_selected_df()
sel_stn

## 4. Get parameters for selected stations and time period

In [8]:
# Get parameters
par_df = ndb_queries.get_station_parameters2(sel_stn, st_dt, end_dt, ora_eng)

# Convert to qgrid
par_grid = qgrid.QgridWidget(df=par_df, show_toolbar=False)

print '%s parameters available.' % len(par_df)
par_grid

66 parameters available.


QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

In [9]:
# Show selected parameters
sel_par = par_grid.get_selected_df()
sel_par

,parameter_id,parameter_name,unit
55,9,TOTN,µg/l
57,11,TOTP,µg/l


## 5. Get water chemistry

In [10]:
# Get water chemsitry
wc_df, dup_df = ndb_queries.get_chemistry_values2(sel_stn, sel_par, 
                                                  st_dt, end_dt, 
                                                  lod_flags, ora_eng)

print 'Number of duplicated records:', len(dup_df)
print 'Number of unique samples:', len(wc_df)

Number of duplicated records: 0
Number of unique samples: 3516


In [ ]:
# Convert duplicates to qgrid
dup_grid = qgrid.QgridWidget(df=dup_df, show_toolbar=False)
dup_grid

In [11]:
# Convert chem to qgrid
wc_grid = qgrid.QgridWidget(df=wc_df, show_toolbar=False)
wc_grid

QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)